In [28]:
import yfinance as yf
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

#Tesla Data and Tesla Revenue

In [24]:
tesla = yf.Ticker('TSLA')
tesla_data = tesla.history(period='max')
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


In [5]:
url = 'https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue'
t_html = requests.get(url).text
tesla_html = BeautifulSoup(t_html, 'html.parser')

In [37]:
Date = np.array([])
Revenue = np.array([])
for r in tesla_html.find_all('tbody')[1].find_all('tr'):
  c = r.find_all('td')
  Date = np.append(Date, c[0].text)
  rev = c[1].text.replace(",", "").replace("$", "")
  if rev == "":
    rev = np.NaN
  Revenue = np.append(Revenue, float(rev))
tesla_revenue = pd.DataFrame({"Date":Date, "Revenue":Revenue})
tesla_revenue.dropna(inplace=True)
tesla_revenue.reset_index(inplace=True)
tesla_revenue.tail()

,index,Date,Revenue
50,50,2010-09-30,31.0
51,51,2010-06-30,28.0
52,52,2010-03-31,21.0
53,54,2009-09-30,46.0
54,55,2009-06-30,27.0


#GameStop Data and GameStop Revenue

In [26]:
game_Stop = yf.Ticker('GME')
game_Stop_data = game_Stop.history(period='max')
game_Stop_data.reset_index(inplace=True)
game_Stop_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716073,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578048,1.607505,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662209,1.603295,1.662209,6892800,0.0,0.0


In [8]:
url = 'https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue'
g_html = requests.get(url).text
gs_html = BeautifulSoup(g_html, 'html.parser')

In [36]:
Date = np.array([])
Revenue = np.array([])
for r in gs_html.find_all('tbody')[1].find_all('tr'):
  c = r.find_all('td')
  Date = np.append(Date, c[0].text)
  rev = c[1].text.replace(",", "").replace("$", "")
  Revenue = np.append(Revenue, rev)
gm_revenue = pd.DataFrame({"Date":Date, "Revenue":Revenue})
gm_revenue.tail()

,Date,Revenue
52,2010-01-31,3524
53,2009-10-31,1835
54,2009-07-31,1739
55,2009-04-30,1981
56,2009-01-31,3492


#Building Dashbords

In [33]:
#This function was borrowed from course, according to course we do not have to know about this.
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("Historical Share Price", "Historical Revenue"), vertical_spacing = .3)
    fig.add_trace(go.Scatter(x=pd.to_datetime(stock_data.Date, infer_datetime_format=True), y=stock_data.Close.astype("float"), name="Share Price"), row=1, col=1)
    fig.add_trace(go.Scatter(x=pd.to_datetime(revenue_data.Date, infer_datetime_format=True), y=revenue_data.Revenue.astype("float"), name="Revenue"), row=2, col=1)
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(showlegend=False,
    height=900,
    title=stock,
    xaxis_rangeslider_visible=True)
    # fig.show()
    plotly.offline.plot(fig, filename='/dir/%s.html' % stock) #save as a html file

In [ ]:
make_graph(tesla_data, tesla_revenue, 'Tesla Stock')

In [ ]:
make_graph(game_Stop_data, gm_revenue, 'GameStop Stock')